In [32]:
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
from heapq import nlargest

In [33]:
# Завантаження англійської моделі SpaCy
nlp = spacy.load("en_core_web_sm")

# Завантаження необхідних ресурсів NLTK
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/sergio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sergio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
# --- Текст для обробки ---
text = """The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound.
"""

In [35]:
# --- 1. Токенізація речень ---
# Використовуємо NLTK, щоб розбити текст на окремі речення
sentence_tokens = sent_tokenize(text)

In [36]:
# --- 2. Лематизація та токенізація слів з SpaCy ---
doc = nlp(text)

In [37]:
# --- 3. Список стоп-слів та пунктуації ---
stop_words = set(stopwords.words("english"))
punctuation_symbols = set(punctuation + "\n\r\t ")

In [38]:
# --- 4. Розрахунок частоти слів ---
word_frequencies = {}
for token in doc:
    # Перевірка: слово не є стоп-словом та не є пунктуацією
    if (
        token.text.lower() not in stop_words
        and token.text.lower() not in punctuation_symbols
    ):
        # Лематизоване слово беремо як ключ
        word = token.lemma_.lower()
        if word not in word_frequencies:
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

In [39]:
# --- 5. Нормалізація частот ---
# Ділимо на найбільше значення, щоб отримати вагу в діапазоні 0..1
max_freq = max(word_frequencies.values())
for word in word_frequencies:
    word_frequencies[word] /= max_freq

In [40]:
# --- 6. Оцінка ваги речень ---
sentence_scores = {}
for sent in sentence_tokens:
    sent_doc = nlp(sent)  # обробка кожного речення в SpaCy
    for token in sent_doc:
        lemma = token.lemma_.lower()
        if lemma in word_frequencies:
            if sent not in sentence_scores:
                sentence_scores[sent] = word_frequencies[lemma]
            else:
                sentence_scores[sent] += word_frequencies[lemma]
sentence_scores

{'The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering.': 6.692307692307692,
 'Because it has achieved significance within the past fifty years, Criteria Consideration G applies.': 0.7692307692307692,
 'Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA.': 7.076923076923076,
 'Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station.': 2.6153846153846154,
 'Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine time

In [41]:
# --- 7. Формування summary ---
select_length = len(sentence_tokens)
summary_sentences = nlargest(select_length, sentence_scores, key=sentence_scores.get)

In [44]:
print("----- Summary -----")
for sentence in summary_sentences:
    print(sentence)
    print(' ')

----- Summary -----
According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials.
 
Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA.
 
Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions.
 
The Orbiter Discove